# U.S. Geological Survey Class GW3099
Advanced Modeling of Groundwater Flow (GW3099)\
Boise, Idaho\
September 16 - 20, 2024

![title](../../../images/ClassLocation.jpg)

## Prototyping Observations for PEST++ with the MV Model

In this notebook, we prototype how to assemble the observations into a single vector and write an instruction file to convey model results to PEST++. The logic of this notebook must get incorporated into the forward run script which we do in the next notebook.

In [ ]:
from pathlib import Path

import pandas as pd

In [ ]:
# select the folder containing example model observation output
rundir = Path("../pest_obs_prototype/")

In [ ]:
# now list the output files - we happen to know the only CSV files
# are the model output files in this dir
outfiles = sorted(list(rundir.glob("*.csv")))
outfiles

In [ ]:
# 👆👆👆👆 note that we were explicit about order here,
# because glob may differ by os (IMPORTANT!)
# So using sorted is important


In [ ]:
# read all the output files into a single dataframe
obs = pd.concat([pd.read_csv(i).T.iloc[1:] for i in outfiles])
obs.columns = ["obsval"]
obs

In [ ]:
# make the DS observation cumulative, including Pollack Ford
obs.loc["DS"] = obs.loc["DS"] + obs.loc["PF"]
# also calculate a few (3) vertical head difference targets
obs.loc["UW02", "obsval"] = obs.loc["U02", "obsval"] - obs.loc["W02", "obsval"]
obs.loc["UW08", "obsval"] = obs.loc["U08", "obsval"] - obs.loc["W08", "obsval"]
obs.loc["UW15", "obsval"] = obs.loc["U15", "obsval"] - obs.loc["W15", "obsval"]

In [ ]:
# write out all these data to an example file
obs.to_csv("../pest_background_files/allobs.dat", sep=" ")

In [ ]:
# make the observation names into lower case
obsnames = [i.lower() for i in obs.index.tolist()]
obsnames

### making the assumption that model output will be concatenated exactly as done above, we can make a simple instruction file that will be used by PEST++ to read model output

In [ ]:
with open("../pest_background_files/allobs.dat.ins", "w") as ofp:
    ofp.write("pif ~\n")
    obs0 = obsnames.pop(0)
    ofp.write(f"l2 w !{obs0}!\n")
    [ofp.write(f"l1 w !{i}!\n") for i in obsnames]